## [--- Day 12: Secret Entrance ---](https://adventofcode.com/2025/day/12)

In [ ]:
import re
from dataclasses import dataclass

RANGE_6 = (0, 1, 2, 3, 4, 5)


@dataclass
class Grid:
    width: int
    height: int
    cells: int
    requirements: tuple[int, int, int, int, int, int]
    left_mask: int
    """bitmask representing the left edge of the grid"""
    right_mask: int
    """bitmask representing the right edge of the grid"""
    full_mask: int
    """bitmask representing the full grid"""
    checkerboard: int
    """bitmask representing the checkerboard pattern of the grid"""
    current: int = 0
    """bitmask representing the current state of the grid"""


def compute_checkerboard_mask(width: int, height: int) -> int:
    mask = 0
    for y in range(height):
        for x in range(width):
            if (x + y) % 2 == 0:
                mask |= 1 << (y * width + x)
    return mask


def build_grids(data: list[re.Match]) -> list[Grid]:
    grids: list[Grid] = []
    for _, dimensions, reqs in data:
        shape_requirements = tuple(map(int, reqs.split()))
        if len(shape_requirements) != 6:
            raise ValueError("Each requirement must have exactly 6 integers.")
        x, y = map(int, dimensions.split("x"))
        grids.append(
            Grid(
                width=x,
                height=y,
                cells=x * y,
                requirements=shape_requirements[0:6],
                left_mask=sum(1 << (r * x) for r in range(y)),
                right_mask=sum(1 << (r * x + (x - 1)) for r in range(y)),
                full_mask=(1 << (x * y)) - 1,
                checkerboard=compute_checkerboard_mask(x, y),
            )
        )

    return grids


def build_shapes(raw_shapes: list[re.Match]) -> list[set[tuple[int, int, int]]]:
    all_shapes: list[set[tuple[int, int, int]]] = []
    for match in raw_shapes:
        raw_shape = parse_shape(match[0].splitlines())
        shape_set: set[tuple[int, int, int]] = set([raw_shape, reflect(raw_shape)])

        # Generate all rotations and reflections
        for _ in (1, 2, 3):
            raw_shape = rotate(raw_shape)
            shape_set.add(raw_shape)
            shape_set.add(reflect(raw_shape))

        all_shapes.append(shape_set)

    return all_shapes


def parse_shape(shape: list[str]) -> tuple[int, int, int]:
    return (
        1 * (shape[0][0] == "#") | 2 * (shape[0][1] == "#") | 4 * (shape[0][2] == "#"),
        1 * (shape[1][0] == "#") | 2 * (shape[1][1] == "#") | 4 * (shape[1][2] == "#"),
        1 * (shape[2][0] == "#") | 2 * (shape[2][1] == "#") | 4 * (shape[2][2] == "#"),
    )


def rotate(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        (((shape[0] & 1) << 2) | ((shape[1] & 1) << 1) | (shape[2] & 1)),
        (((shape[0] & 2) << 1) | (shape[1] & 2) | ((shape[2] & 2) >> 1)),
        ((shape[0] & 4) | ((shape[1] & 4) >> 1) | ((shape[2] & 4) >> 2)),
    )


def reflect(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        ((shape[0] & 1) << 2) | (shape[0] & 2) | ((shape[0] & 4) >> 2),
        ((shape[1] & 1) << 2) | (shape[1] & 2) | ((shape[1] & 4) >> 2),
        ((shape[2] & 1) << 2) | (shape[2] & 2) | ((shape[2] & 4) >> 2),
    )


def print_shape(shape: tuple[int, int, int]) -> None:
    grid = (
        ("# " if (shape[y] & x) != 0 else ". " for x in (1, 2, 4)) for y in (0, 1, 2)
    )

    print("\n".join("".join(row) for row in grid))
    print("-" * 5)


def compute_shape_parity(
    shapes: list[set[tuple[int, int, int]]],
) -> list[tuple[int, int]]:
    parity_reqs: list[tuple[int, int]] = []
    checker_board = 0b101_010_101  # 3x3 checkerboard pattern
    for variants in shapes:
        min_b, min_w = 9, 9  # max 9 cells in shape
        for shape in variants:
            mask = shape[0] | (shape[1] << 3) | (shape[2] << 6)
            black_1 = (mask & checker_board).bit_count()
            white_1 = (mask & ~checker_board).bit_count()

            black_2 = (mask & ~checker_board).bit_count()
            white_2 = (mask & checker_board).bit_count()

            min_b = min(min_b, black_1, black_2)
            min_w = min(min_w, white_1, white_2)

        parity_reqs.append((min_b, min_w))
    return parity_reqs


def create_placements(
    grid: Grid, shapes: list[set[tuple[int, int, int]]]
) -> list[list[list[int]]]:
    """
    Generates all valid grid masks for shape variants, indexed by their 'anchor bit'
    (the lowest set bit in the mask).

    Since every shape has a 3x3 footprint with no empty rows, the anchor is
    guaranteed to be in the first row.

    This enables a 'First Empty Bit' search strategy: the solver finds the first
    hole in the grid and only checks masks whose anchor matches that specific bit.
    """
    W, H = grid.width, grid.height
    placements: list[list[list[int]]] = [[[] for _ in RANGE_6] for _ in range(W * H)]

    for shape_idx, variants in enumerate(shapes):
        for shape in variants:
            # lowest set bit for this shape
            for y in range(H - 2):
                y0_W = y * W
                y1_W = (y + 1) * W
                y2_W = (y + 2) * W
                for x in range(W - 2):
                    mask = (
                        shape[0] << (x + y0_W)
                        | shape[1] << (x + y1_W)
                        | shape[2] << (x + y2_W)
                    )
                    anchor = (mask & -mask).bit_length() - 1
                    placements[anchor][shape_idx].append(mask)
    return placements


def check_waste(grid: Grid, target_bit: int, min_size: int) -> int | None:
    empty_space = ~grid.current & grid.full_mask
    flood = 1 << target_bit
    width = grid.width
    width_2 = width * 2

    while True:
        next_flood = (
            flood
            | (
                ((flood << 1) & ~grid.right_mask)  # left
                | ((flood >> 1) & ~grid.left_mask)  # right
                | (flood << width)  # down
                | (flood >> width)  # up
            )
            & empty_space
        )

        if next_flood == flood:
            return flood  # return the waste
        flood = next_flood

        if flood.bit_count() >= min_size:
            if flood & (flood >> width) & (flood >> width_2):
                shift_1 = (flood & ~grid.left_mask) >> 1
                shift_2 = (shift_1 & ~grid.left_mask) >> 1
                if flood & shift_1 & shift_2:
                    return None  # Viable for 3x3 shape


def solve_grid(
    grid: Grid,
    requirements: list[int],
    placements: list[list[list[int]]],
    area_needed: int,
    shape_weights: tuple[int, ...],
    shape_parities: list[tuple[int, int]],
    visited: set[tuple[int, tuple[int, ...]]],
) -> bool:
    current = grid.current
    tuple_reqs = tuple(requirements)
    state = (current, tuple_reqs)

    # --- Memoisation Prune ---
    # Has this state been visited before?
    if state in visited:
        return False

    # Success case
    if area_needed == 0:  # all shapes placed
        return True

    # --- Global Pruning ---
    # Is there enough space left?
    remaining_space = grid.cells - current.bit_count()
    slack = remaining_space - area_needed
    if area_needed > remaining_space or slack < 0:  # not enough space left
        visited.add(state)
        return False

    # --- Parity Pruning ---
    # Are there enough black/white cells left?
    empty_cells = ~current & grid.full_mask
    remaining_black = (empty_cells & grid.checkerboard).bit_count()
    remaining_white = (empty_cells & ~grid.checkerboard).bit_count()

    required_black = 0
    required_white = 0
    for i in RANGE_6:
        if requirements[i] > 0:
            shape_black, shape_white = shape_parities[i]
            required_black += shape_black * requirements[i]
            required_white += shape_white * requirements[i]

    if required_black > remaining_black or required_white > remaining_white:
        visited.add(state)
        return False

    # --- Select Target Bit ---
    target_bit = (current ^ (current + 1)).bit_length() - 1

    # --- Out of Bounds Pruning ---
    # Is the grid exhausted?
    if target_bit >= grid.cells:
        visited.add(state)
        return False

    # --- Island and Geometric Pruning ---
    # Can this island fit any remaining shapes?
    min_shape_weight = min(shape_weights[i] for i in RANGE_6 if requirements[i] > 0)

    waste_mask = check_waste(grid, target_bit, min_shape_weight)
    if waste_mask is not None:
        waste_size = waste_mask.bit_count()
        if waste_size <= slack:
            grid.current = current | waste_mask
            if solve_grid(
                grid,
                requirements,
                placements,
                area_needed,
                shape_weights,
                shape_parities,
                visited,
            ):
                return True

        # Backtrack, the waste exceeds slack or fails to resolve
        grid.current = current
        visited.add(state)
        return False

    # Try to fill the island with shapes
    for shape_id in RANGE_6:
        if requirements[shape_id] == 0:
            continue
        for mask in placements[target_bit][shape_id]:
            if not (current & mask):
                grid.current = current | mask
                requirements[shape_id] -= 1

                if solve_grid(
                    grid,
                    requirements,
                    placements,
                    area_needed - shape_weights[shape_id],
                    shape_weights,
                    shape_parities,
                    visited,
                ):
                    return True

                # Backtrack
                requirements[shape_id] += 1
                grid.current = current

    # Try skipping this bit, if there's slack
    if slack > 0:
        grid.current = current | (1 << target_bit)
        if solve_grid(
            grid,
            requirements,
            placements,
            area_needed,
            shape_weights,
            shape_parities,
            visited,
        ):
            return True

        # Backtrack
        grid.current = current

    visited.add(state)
    return False


def solve() -> None:
    with open("..\\data\\12 example.txt") as file:
        pattern = re.compile(r"^([#.\n]{11})|(\d+x\d+): ((?:\d+ ?){6})$", re.MULTILINE)
        data = pattern.findall(file.read())

    raw_shapes = data[:6]
    shapes: list[set[tuple[int, int, int]]] = build_shapes(raw_shapes)
    shape_weights: tuple[int, ...] = tuple(
        sum(row.bit_count() for row in list(s)[0]) for s in shapes
    )
    shape_parities = compute_shape_parity(shapes)
    grids: list[Grid] = build_grids(data[6:])
    results: list[tuple[Grid, bool]] = []

    for grid in grids:
        area_needed = sum(shape_weights[i] * grid.requirements[i] for i in RANGE_6)

        if area_needed > grid.cells:  # unsolveable
            results.append((grid, False))
            print(
                f"Tested grid {grid.width}x{grid.height}: False (trivially unsolvable)"
            )
            continue

        placements = create_placements(grid, shapes)
        solved = solve_grid(
            grid,
            list(grid.requirements),
            placements,
            area_needed,
            shape_weights,
            shape_parities,
            set(),
        )
        results.append((grid, solved))
        print(f"Tested grid {grid.width}x{grid.height}: {solved}")

    count = 0
    for grid, solvable in results:
        if solvable:
            count += 1
    print(f"Total solvable grids: {count}/{len(grids)}")


solve()

Tested grid 4x4: False
Tested grid 12x5: True
Tested grid 12x5: False
Total solvable grids: 1/3
